# IMPORT PACKAGES

In [ ]:
import glob
import pandas as pd
import math
import sklearn
from yellowbrick.model_selection import LearningCurve
import pandas as pd
import numpy as np
from scipy import interp
import shap 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import cycle
from pandas import DataFrame
from numpy import matrix 
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
import joblib
from sklearn.metrics import roc_curve, auc, roc_auc_score,accuracy_score,f1_score,recall_score,precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import Normalizer

# READ MATRICES

Pearson's connection matrix of the EEG experiments of subjects who ingested Ayahuasca at the time after the psychedelic activation time can be found in: Alves, Caroline (2022): With-ayahuasca. figshare. Dataset. https://doi.org/10.6084/m9.figshare.21082513.v1

Pearson's connection matrix of the EEG experiments of subjects who ingested Ayahuasca at the time before the psychedelic activation time: Alves, Caroline (2022): No-ayahuasca. figshare. Dataset.
https://doi.org/10.6084/m9.figshare.21082531.v1

In [ ]:
## Define function to read
def pixels_from_path(file_path):
    im = open(file_path)
    test = pd.read_csv(im,sep=",", index_col=False,header=None)
    #im = im.resize(IMG_SIZE)
    np_im = np.array(test)
    #matrix of pixel RGB values
    return np_im

print("with ayahuasca")
#insert path for Pearson connectivity matrix with ayahuasca
#aya = np.asarray([pixels_from_path(n) for n in glob.glob(r'PATH-ayahuasca-EEG-time-series/*csv')])
aya = np.asarray([pixels_from_path(n) for n in glob.glob(r'connectivity-matrix/with-Ayahuasca/*csv')])
print("shape for ayahuasca matrices")
print(np.shape(aya ))
#insert path for Pearson connectivity matrix no ayahuasca
print("no ayahuasca")
#normal = np.asarray([pixels_from_path(n) for n in glob.glob(r'PATH-no-ayahuasca-EEG-time-series/*csv')])
normal = np.asarray([pixels_from_path(n) for n in glob.glob(r'connectivity-matrix/no-Ayahuasca/*csv')])
#print(normal)
print("shape for no ayahuasca matrices")
print(np.shape(normal))


# PREPARING TRAIN SET AND LABELS

In [ ]:
## Preparing train set
scaler = StandardScaler()
x = np.concatenate([normal ,aya])

X= np.concatenate(x, axis=0)
X=scaler.fit_transform(X)
X=np.nan_to_num(X) 
print("shape for train set")
print(np.shape(X))
## Preparing test set
yes_l = [1]*len(np.concatenate(aya, axis=0))
no_l = [0]*len(np.concatenate(normal, axis=0))
labels = yes_l + no_l
#print(labels) ## the labels of the graphs
y=np.nan_to_num(labels)
print("length for test set")
print(len(y))

# DEFINE MACHINE LEARNING MODEL, RESAMPLING AND GRID SEARCH TUNNING HYPERPARAMETERS 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.25)
def get_mean_measures(model, X, y): 
    global param_grid
    global model_est
    #######
    if model == 'RF':
        param_grid = {
            #'max_depth': [1,2,5,10,20,80],
   # 'max_features': [1,2, 3,5,10],
   # 'min_samples_leaf': [1,2,3, 4, 5],
  #  'min_samples_split': [1,2,8, 10, 12,20],
    'n_estimators': range(1, 50,2)
    #'splitter':['best','random']
  }
        model_est= RandomForestClassifier(random_state=1234)
    ########
    elif model == 'SVM':
        param_grid = {'C':range(1, 30)},
        model_est= SVC(probability=True,random_state=1234)         
    ########
    elif model == 'NB':
        param_grid=  {'var_smoothing': np.logspace(0,-9, num=100)}
        model_est= GaussianNB()
    
    #########
    elif model == 'MLP':
        param_grid= {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)]}
  #  'activation': ['tanh', 'relu'],
   # 'solver': ['sgd', 'adam'],
   # 'alpha': [0.0001, 0.05],
   # 'learning_rate': ['constant','adaptive']}
        model_est=MLPClassifier(random_state=1234)
    #########
    
    elif model == 'KNN':
        param_grid= {
   # 'n_neighbors': (1,10, 1),
    #'leaf_size': (20,40,1),
    #'p': (1,2),
    #'weights': ('uniform', 'distance'),
    #'metric': ('minkowski', 'chebyshev') 
            'n_neighbors': range(1, 30)
        }
        model_est= KNeighborsClassifier()
    #####
    elif model == 'LR':
        param_grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
        model_est= LogisticRegression(random_state=1234)
    #####
    elif model== 'XGboost':
        param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
        model_est= XGBClassifier(random_state=1234)
    
    ##########
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 1234) 
    recall = list()
    precision = list()
    f = list()
    accuracy=list()
    roc=list()
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
ps
    for train_index, test_index in skf.split(X,y):
        xtr, xvl = X.loc[train_index],X.loc[test_index]
        ytr, yvl = y.loc[train_index],y.loc[test_index]

        # Corrigindo o formato do ytr para ajustar o modelo
        ytr = ytr.to_numpy()
        ytr = ytr.reshape(len(ytr),)
        
        clf = GridSearchCV(model_est, param_grid, scoring='roc_auc')
        clf.fit(xtr,ytr)
        # Ajuste do modelo
     #   model.fit(xtr,ytr)
        clf_best= clf.best_estimator_
        # predição
        y_pred = clf.best_estimator_.predict(xvl)

        # Corigindo o formato do yvl para poder calcular a acurácia
        yvl = yvl.to_numpy()
        yvl = yvl.reshape(len(yvl),)

        # Calculando a acurácia, recall, precision, f1
        roc.append(roc_auc_score(yvl,y_pred,average='weighted'))
        accuracy.append(accuracy_score(yvl, y_pred))
        recall.append(recall_score(yvl,y_pred,average='weighted'))
        precision.append(precision_score(yvl,y_pred,average='weighted'))
      #  print(f1_score(yvl,y_pred, average='weighted'))
        f.append(sklearn.metrics.f1_score(yvl,y_pred,average='weighted'))
        
    return(roc,np.mean(roc), np.std(roc),        
           accuracy,np.mean(accuracy), np.std(accuracy),
           recall,np.mean(recall), np.std(recall),
            precision,np.mean(precision), np.std(precision),
           f,np.mean(f),np.std(f),
           clf_best)
    #return(np.mean(accuracy), np.mean(recall), np.mean(precision), np.mean(f),clf_best)


# RESAMPLING, CALLING THE MODEL AND PRINT RESULTS

In [ ]:
#'RF'- Random forest model
#'SVM'- Support Vector Machine model
#'NB'- Naive Bayes model
#'MLP'- Multilayer perceptron model
#'KNN'- k-nearest neighbors model
#'LR'- Logistic regression model
#'XGboost'- Extreme Gradient Boosting classifier model

roc,roc_mean, std_roc,acc,accuracy_mean, std_accuracy, rec,recall_mean, std_recall, pre,precision_mean,std_precision,f, f_mean, std_f, clf = get_mean_measures('SVM',X_train,y_train)

results = { 
            'AUC': [roc_mean],
            'Accuracy' : [accuracy_mean],
            'Recall' : [recall_mean], 
            'Precision' : [precision_mean],
            'f1-Score' : [f_mean]}
df_results = pd.DataFrame(results)
print('Train set preformance')
print(df_results)

print('Test set preformance')
y_pred_test =  clf.predict(X_test)
print('AUC test:',   roc_auc_score(y_test,y_pred_test))
print('Accuracy test:', accuracy_score(y_test,y_pred_test))
print('F1 score test:', f1_score(y_test,y_pred_test, average="macro", pos_label=0))
print('Recall test:', recall_score(y_test,y_pred_test,average="macro", pos_label=0))
print('Precision test:', precision_score(y_test,y_pred_test,average="macro", pos_label=0))
print(classification_report(y_test,y_pred_test))

In [ ]:
df_results 

# PLOT CONFUSION MATRIX

In [ ]:
print('Confusion matrix')
cm = confusion_matrix(y_test, y_pred_test)
cm_sum = np.sum(cm, axis=1, keepdims=True)
cm_perc = cm / cm_sum.astype(float) * 100
annot = np.empty_like(cm).astype(str)
nrows, ncols = cm.shape
for i in range(nrows):
    for j in range(ncols):
        c = cm[i, j]
        p = cm_perc[i, j]
        if i == j:
            s = cm_sum[i]
            annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
        elif c == 0:
            annot[i, j] = ''
        else:
            annot[i, j] = '%.1f%%\n%d' % (p, c)
cm = pd.DataFrame(cm)
   # print(cm)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
#fig, ax = plt.subplots(figsize=figsize)
sns.heatmap(cm, annot=annot, fmt='',cmap='rocket_r')


# PLOT ROC CURVE

In [ ]:
print('Roc curve')
n_classes =2

t1=sum(x==0 for x in y_pred_test-y_test)/len(y_pred_test)

    ### MACRO
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(np.array(pd.get_dummies(y_test))[:, i], np.array(pd.get_dummies(y_pred_test))[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

lw=2
plt.figure(figsize=(8,5))
plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve'
               ''.format(roc_auc["macro"]),
         color='slategray', linestyle=':', linewidth=4)

colors = cycle(['purple', 'lightseagreen'])
for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0}'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--',color='#cb416b', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.annotate(' Random Guess',(.5,.48),color='#cb416b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Receiver Operating Characteristic for Suport Vector Machine')
plt.legend(loc="lower right")
#plt.savefig("ROC-svm-measures.pdf",dpi=300, bbox_inches = "tight")
plt.show()
    

# PLOT LEARNING CURVE

In [ ]:
sizes = np.linspace(0.3, 1.0, 10)
#best= clf.best_estimator_
visualizer = LearningCurve(
   clf, cv=10, scoring='accuracy', train_sizes=sizes, n_jobs=4
)

visualizer.fit(X_train, y_train)
# Fit the data to the visualizer
visualizer.show()  